In [1]:
library(itecv) # should be loaded first
library(magrittr)
library(foreach)
library(doFuture)
library(zeallot)
library(tidyverse)
library(rlearner)

plotsize = function (x,y) options(repr.plot.width=x, repr.plot.height=y)

registerDoFuture()
plan(cluster, workers = 4)

Loading required package: future
Loading required package: iterators
Loading required package: parallel

Attaching package: ‘zeallot’

The following objects are masked from ‘package:future’:

    %->%, %<-%

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 2.2.1     ✔ purrr   0.2.5
✔ tibble  1.4.2     ✔ dplyr   0.7.6
✔ tidyr   0.8.1     ✔ stringr 1.3.1
✔ readr   1.1.1     ✔ forcats 0.3.0
Warning message:
“package ‘dplyr’ was built under R version 3.5.1”── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ purrr::accumulate() masks foreach::accumulate()
✖ tidyr::extract()    masks magrittr::extract()
✖ dplyr::filter()     masks stats::filter()
✖ dplyr::lag()        masks stats::lag()
✖ purrr::set_names()  masks magrittr::set_names()
✖ purrr::when()       masks foreach::when()
Warning message:
“replacing previous import ‘caret::lift’ by ‘purrr::lift’ when loading ‘rlearner’”Warning message:
“replacing previous import 

In [2]:
model_specs = list(
    gbm = list(
        tune_grid = expand.grid(
            n.trees = seq(1,501,40), 
            interaction.depth=3, 
            shrinkage = 0.1, 
            n.minobsinnode=3),
        extra_args = list(
            verbose=F, 
            bag.fraction=0.5)),
    glmnet = list(
        tune_grid = expand.grid(
            alpha=c(0,0.5,1),
            lambda=exp(seq(-5,2,0.4))),
        extra_args = list()))

metrics = list(
    mse = bundle_mse,
    wmse = bundle_wmse,
#     dr_mse = bundle_dr_mse,
    r_objective = bundle_r_objective,
    match_mse = bundle_match_mse,
    trans_mse = bundle_trans_mse,
#     match_mdc = bundle_match_mdc,
#     trans_mdc = bundle_trans_mdc,
    ip_value = bundle_ip_value,
    dml_value = bundle_dml_value,
#     gain = bundle_gain,
#     c_benefit = bundle_c_benefit,
#     qini = bundle_qini,
    random = bundle_random)

replicates = 1:2
DGPs = powers_DGPs() %>% list(1:16) %>% 
    pmap(function(dgp, id) {
        dgp$id = id
        dgp
    })
c(itrain, ival, itest) %<-% make_indices(100, 100, 100)

In [3]:
learners = list(
     rlearner=rlearner_cv, 
     slearner=slearner_cv, 
     tlearner=tlearner_cv) 

In [4]:
run_sim = function(replicate, DGP, model_specs, learners, metrics, itrain, ival, itest) {
    data = create_data(DGP, length(c(itrain, ival, itest)))
    estimates = estimate_val_test(data, itrain, model_specs)
    val_bundle = learn_validation_auxiliaries(data, ival, model_specs, randomized=DGP$randomized)
    val_metrics = estimate_val_metrics(estimates, val_bundle, metrics, ival)
    learner_estimates = estimate_learner_test(data, learners, model_specs, itrain, ival, itest)
    test_metrics = calc_test_metrics(data, bind_rows(estimates, learner_estimates), itest)
    
    inner_join(val_metrics, test_metrics, by="model") %>%
        mutate(replicate=replicate, dgp=DGP$id)
}

In [5]:
start_time <- Sys.time()

foreach(replicate = replicates, 
        .export=c("model_specs", "learners", "metrics", "itrain", "ival", "itest", "run_sim"),
        .combine = bind_rows,
        .packages = (.packages())) %:%
    foreach(DGP = DGPs, 
            .export=c("model_specs", "learners", "metrics", "itrain", "ival", "itest", "run_sim"),
            .combine = bind_rows,
            .packages = (.packages()))  %dopar%  {
        run_sim(replicate, DGP, model_specs, learners, metrics, itrain, ival, itest) 
    } %>%
write_csv("data/experimental_results_test.csv")

end_time <- Sys.time()
end_time - start_time

ERROR: Error in {: task 1 failed - "could not find function "estimate_learner_test""
